In [ ]:
!docker run -p 5000:5000 irony/caffe-docker-classifier

In [ ]:
!curl http://localhost:5000/classify_url?imageurl=http://www.atlantalabrescue.com/wp-content/uploads/2017/08/buddy-lead-400x200.jpg

In [6]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

Starting local Kubernetes v1.9.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [7]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io "kube-system-cluster-admin" created


In [8]:
!helm init

$HELM_HOME has been configured at /Users/tejdeepkautharam/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [9]:
!helm install ../seldon-core/helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../seldon-core/helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Tue Jun 26 15:49:07 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Tue Jun 26 15:49:08 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVA

In [10]:
!kubectl create -f deploy.json

seldondeployment.machinelearning.seldon.io "caffe-class" created


In [28]:
!kubectl get seldondeployments caffe-class -o jsonpath='{.status}' 

map[predictorStatus:[map[name:caffe-class-single-model replicas:1 replicasAvailable:0]]]

In [26]:
!cd ../seldon-core/util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


In [31]:
!python ../seldon-core/util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

Traceback (most recent call last):
  File "../seldon-core/util/api_tester/api-tester.py", line 210, in <module>
    run(args)
  File "../seldon-core/util/api_tester/api-tester.py", line 140, in run
    contract = json.load(open(args.contract,'r'))
IOError: [Errno 2] No such file or directory: 'contract.json'


In [32]:
!minikube delete

Deleting local Kubernetes cluster...
Machine deleted.
